# Sample for KFServing SDK 

This is a sample for KFServing SDK. 

The notebook shows how to use KFServing SDK to create, get, patch and delete InferenceService.

In [ ]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements

## Define InferenceService

Firstly define the model spec for default model spec, and then define the inferenceservice basic on the model spec.

In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='gs://kfserving-samples/models/tensorflow/flowers',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='flower-sample', namespace='kubeflow'),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

## Create InferenceService

Call KFServingClient to create InferenceService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)

## Check the InferenceService

In [ ]:
KFServing.get('flower-sample', namespace='kubeflow', watch=True, timeout_seconds=120)

## Add Canary to InferenceService

Patch the InferenceService to add canary model.

In [ ]:
canary_endpoint_spec = V1alpha2EndpointSpec(
                         predictor=V1alpha2PredictorSpec(
                           tensorflow=V1alpha2TensorflowSpec(
                             storage_uri='gs://kfserving-samples/models/tensorflow/flowers-2',
                             resources=V1ResourceRequirements(
                                 requests={'cpu':'100m','memory':'1Gi'},
                                 limits={'cpu':'100m', 'memory':'1Gi'}))))

isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='flower-sample', namespace='kubeflow'),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec,
                                                     canary=canary_endpoint_spec,
                                                     canary_traffic_percent=10))

KFServing.patch('flower-sample', isvc)

In [ ]:
KFServing.get('flower-sample', namespace='kubeflow', watch=True, timeout_seconds=120)

## Promote Canary to Default

In [ ]:
KFServing.promote('flower-sample', namespace='kubeflow')

In [ ]:
KFServing.get('flower-sample', namespace='kubeflow', watch=True, timeout_seconds=120)

## Delete the InferenceService

In [ ]:
KFServing.delete('flower-sample', namespace='kubeflow')